# DataFrames and data wrangling

In this notebook we will look at the [DataFrames.jl](https://juliadata.github.io/DataFrames.jl/stable/) package.

`DataFrame` objects contain data tables consisting of a series of vectors, each representing a column or variable.

We will focus on the following operations:

* Filtering rows
* Selecting columns
* Adding and modifying columns
* Sorting
* Performing caculations on all rows or by groups of rows

The [DataFramesMeta.jl](https://github.com/JuliaData/DataFramesMeta.jl) and [Query.jl](https://www.queryverse.org/Query.jl/stable/) packages provide additional functionality for working with DataFrames. We'll use some functions from Query.jl in this notebook.

As always we first have to load the package.

In [ ]:
using DataFrames

Here is a simple `DataFrame` object created with code.

In [ ]:
df = DataFrame(x = 1:5, y = ["red", "blue", "red", "blue", "green"])

Individual columns can be refenced as `df.y` or `df[!, :y]`. Neither of these makes a copy of the column, so if the data in the column changes, it will change for all references.

In [ ]:
y = df[!, :y]

In [ ]:
y[3] = "purple"

In [ ]:
df

To make a copy of a column, use `df[:, :y]` instead.

In [ ]:
y2 = df[:, :y]

In [ ]:
y2[3] = "yellow"

In [ ]:
df, y2

Notice that the data in `df` did not change this time.

## Missing values

Julia supports representing missing values in the statistical sense that although we don't have data for a particular value, a valid value still theoretically exists. This is similar to `NULL` in SQL and `NA` in R.

Let's look at how missing values propagate.

In [ ]:
missing + 1

In [ ]:
"a" * missing

In [ ]:
cos(missing)

In [ ]:
true | true, true | false, false & true

In [ ]:
missing | true, missing | false, missing & true

In [ ]:
missing == missing # why?

What happens if we have both missing and nonmissing values?

In [ ]:
a = [1, 2, missing, 4]

In [ ]:
using Statistics
mean(a)

We can skip missing values in calculations by using `skipmissing`.

In [ ]:
mean(skipmissing(a))

Or we can replace missing values with some other value such as zero by using `coalesce`. Note that we call `coalesce.(a, 0)` to apply `coalesce` to each element of the array. We'll learn more about what the dot does in the section titled **Dot syntax for vectorizing functions and operators** later in this notebook.

In [ ]:
coalesce.(a, 0)

In [ ]:
mean(coalesce.(a, 0))

Note how the value of the mean is different than when we used `skipmissing`.

Now lets see how missing values work in `DataFrames`.

In [ ]:
df = DataFrame(i = 1:5,
               x = [missing, 4, missing, 2, 1],
               y = [missing, missing, "c", "d", "e"])

First, notice how the types of the columns containing missing data have changed.

Next, we demostrate the `dropmissing` function from the `DataTables` package. This function drops rows of the dataframe that contain any missing values.

In [ ]:
dropmissing(df)

We can also specify which columns to look at for missing values.

In [ ]:
dropmissing(df, :y)

We can still use `skipmissing` on individual columns if we want.

In [ ]:
mean(skipmissing(df[!, :x]))

## NYC flights

For the rest of this notebook, we'll work with the `flights` dataset from the R package `nycflights`.

In [ ]:
using RData
nycflights = load("../data/nycflights13.RData")

We can see descriptive statistics for a DataFrame using the `describe` function.

In [ ]:
flights = nycflights["flights"]
describe(flights)

### Dot syntax for vectorizing functions and operators

Before we learn about filtering, we need to learn about Julia's "dot syntax" for vectorizing functions and operators.

Suppose we have a vector of floating point numbers:

In [ ]:
A = [1.0, 2.0, 3.0]

And suppose we want to calculate the sine of each number. We can calculate the sine of a number using the `sin` function like this:

In [ ]:
sin(1.0)

What happens if we call `sin` on `A`?

In [ ]:
sin(A)

The `sin` function doesn't know how to operate on the type `Array{Float64,1}`, which is how Julia describes a one-dimensional array of double precision (64-bit) floating point numbers.

Some languages would require us to use a separate "vectorized" function, but in Julia, we can do this automatically usiing the following "dot" syntax:

In [ ]:
sin.(A)

The same thing works with operators, but the dot comes before the operator. For example:

In [ ]:
A .^ 3

### Filtering rows

Here is an example of filtering rows. Suppose we want only the rows where `month` equals 7 and `day` equals 17. We use the `first` function to display just the first 6 rows of the result in the notebook.

Notice the use of dot syntax.

In [ ]:
first(flights[(flights.month .== 7) .& (flights.day .== 17), :], 6)

If we wanted rows where `month` is 11 or 12, we could do this. (The output is suppressed by the ; at the end.)

In [ ]:
flights[(flights.month .== 11) .| (flights.month .== 12), :];

The following does the same thing, but requires a little explanation. Here `in([11, 12])` actually returns a *function* that checks whether its argument is *in* the collection `[11, 12]`. This function is then vectorized (or broadcast) over the `month` column of the dataframe using a dot. The result is the rows of the dataframe for which month is 11 or 12. We then display just the first 6 rows.

In [ ]:
first(flights[in([11, 12]).(flights.month), :], 6)

Julia takes a little getting used to, but can be very expressive and powerful.

### Selecting columns

We select columns using the `select` and `select!` functions. The `select` function returns a new dataframe, while `select!` does an in-place select, returning a view into the existing dataframe. We'll just use `select`.

Let's refresh our memory of what columns we have available.

In [ ]:
names(flights)

We can select columns by name like this:

In [ ]:
first(select(flights, [:year, :month, :day]), 6)

We can also select columns by position, but this can easily result in errors if the columns of a dataframe later change.

In [ ]:
first(select(flights, [1, 3, 5]), 6)

There several other ways we can select columns, but a handy one matches column names using a *regular expression*.

In [ ]:
first(select(flights, r"^(dep|arr)"), 6)

### Adding and modifying columns

We'll work with a subset of columns of the `flight` dataset. We could select those columns using standard `DataFrame` functionality like we did above, but this time we will select columns using the `@select` macro from the `Query.jl` package. This package provides functionality similar the `dplyr` package for R.

Notice how we are also using Julia's pipe operator `|>`, which is similar to, *but not the same as*, `dplyr`'s `%>%` operator.

In [ ]:
using Query

flights_sml = flights |> 
  @select(1:3, endswith("delay"), :distance, :air_time) |>
  DataFrame

first(flights_sml, 6)

Suppose we want to add a couple of caculated columns. We could do that using just the `DataFrames` package, but here we will add columns using the `@mutate` macro, also from `Query.jl`.

In [ ]:
flights_sml |>
  @mutate(gain = _.dep_delay - _.arr_delay,
          speed = (_.distance / _.air_time) * 60) |>
  DataFrame |>
  (x -> first(x, 6))

### Sorting

A `DataFrame` can be sorted using the standard `sort` function. Note that `sort` produces a copy of the `DataFrame`. To sort in place, use `sort!` instead.

In [ ]:
first(sort(flights_sml, (:year, :month, :day)), 6)

## Performing calculations on all rows or groups of rows

Since each column of a `DataFrame` is a vector of values, we can perform the usual operations on individual columns. For example, here we calculate the mean departure delay.

In [ ]:
using Statistics
mean(flights_sml.dep_delay)

Oops! Our dataset contains `missing` values. To exclude them, we can use the `skipmissing` function.

In [ ]:
mean(skipmissing(flights_sml.dep_delay))

That worked, but stop and think. What does it mean if a flight departure delay (`dep_delay`) is missing?

If a missing departure delay means an on-time departure, we should treat missing values as zero, like this. Notice that we use the dot operator to apply `coalesce` to each value in the column.

In [ ]:
mean(coalesce.(flights_sml.dep_delay, 0))

The `DataFrames` package also contains an `aggregate` function that we can use on multiple columns.

For example, we can sum each column like this:

In [ ]:
aggregate(flights_sml, sum)

To skip missing values

In [ ]:
aggregate(flights_sml, x->sum(skipmissing(x)))

Does it matter whether we use `skipmissing` or `coalesce` when calculating a sum?

We can calculate means and sums like this:

In [ ]:
aggregate(flights_sml, [sum, x->mean(coalesce.(x, 0))])

The Jupyter notebook doesn't display all columns, but we can fix this by calling the `show` function with `allcols=true`.

In [ ]:
aggregate(flights_sml, [sum, x->mean(coalesce.(x, 0))]) |>
  (df -> show(df, allcols=true))

### Grouping

Suppose we want to calculate mean departure delay by airline. We can do that using the `groupby` and `aggregate` functions.

This example also introduces `@rename` and `@orderby_descending`.

In [ ]:
flights |>
  @select(:carrier, :dep_delay) |>
  DataFrame |>
  (df -> groupby(df, :carrier)) |>
  (df -> aggregate(df, x->mean(coalesce.(x, 0)))) |>
  @rename(:dep_delay_function => :mean_departure_delay) |>
  @orderby_descending(_.mean_departure_delay)

### Final example

As a final example, we will join the results we just computed with the `airlines` dataset to produce a more readable report.

In [ ]:
airlines = nycflights["airlines"]

In [ ]:
flights |>
  @select(:carrier, :dep_delay) |>
  DataFrame |>
  (df -> groupby(df, :carrier)) |>
  (df -> aggregate(df, x->mean(coalesce.(x, 0)))) |>
  @rename(:dep_delay_function => :mean_departure_delay) |>
  @join(airlines, _.carrier, _.carrier, {__.name, _.mean_departure_delay}) |>
  @orderby_descending(_.mean_departure_delay)